 # Track Source Direction with 6C
<img src="pictures/Merapi2006_BaF.png" alt="Merapi Block and Ash Flow 2006" width="600"/>

### Back azimuth estimation using vertical translational and two horizontal rotational ground motions
*Developed by Shihao Yuan (Colorado School of Mines); modified by Sabrina Keil; further modified by J. Wassermann 


<img src="pictures/coord.jpg" alt="drawing" width="300"/>
Figure 1. The right-hand coordinate system. Red arrow indicates the propagation direction of the plane wave with a polar angle ($\theta$) and azimuth ($\varphi$). NOTE: in the practical we are estimating the backazimuth! Clockwise rotations from North or Z are positive while counterclockwise rotations are negative.



### For incident Rayleigh waves:

\begin{align} 
	\mathbf{s} =  \frac{1}{v}
	\begin{bmatrix}
	\sin \varphi \\
	\cos \varphi  \\
	0 \\
	\end{bmatrix},   \quad \quad     
	\mathbf{n} = 
	\begin{bmatrix}
	\sin \xi \sin \varphi \\
	\sin \xi \cos \varphi \\
	\cos \xi \\
	\end{bmatrix}, \quad \quad 
	\mathbf{s} \times \mathbf{n} =  \frac{1}{v} 
	\begin{bmatrix}
	 \cos \xi \cos \varphi  \\
	-\cos \xi \sin \varphi \\
	  0  \\[20pt]
	\end{bmatrix} , 
\end{align}

where the angle $\xi$ denotes ellipticity angle of the Rayleigh wave, determining the eccentricity and the sense of rotation of the particle motion. If $\xi \in (−\pi/2, 0)$, the Rayleigh wave elliptical motion is said to be retrograde. If $\xi \in (0, \pi/2)$ the wave is said to be prograde.


\begin{align}
	\text{Acceleration: } \mathbf{\ddot{u}} =   - \omega^2 A
	& 	\begin{bmatrix}
	\sin \xi \sin \varphi &\cos [ \omega (t - \mathbf{s} \cdot \mathbf{r}) + \phi ] \\
	\sin \xi \cos \varphi &\cos [ \omega (t - \mathbf{s} \cdot \mathbf{r}) + \phi ] \\
	\cos \xi  &\cos [ \omega (t - \mathbf{s} \cdot \mathbf{r}) + \phi + \pi/2] \\
	\end{bmatrix} , \\[10pt]
	\text{Rotational rate: } \mathbf{\dot{\Omega}} =  A \frac{\omega^2}{v}
	& \begin{bmatrix}	
	 \cos \xi \cos \varphi  \\
	-\cos \xi \sin \varphi \\
	  0 \\ 
	\end{bmatrix} \cos [ \omega (t - \mathbf{s} \cdot \mathbf{r}) + \phi + \pi/2], 
\end{align}

where the vertical component acceleration has a $\pi/2$ phase shift compared to the horizontal components, causing elliptical particle motions. 

+ The same as SV waves, using two horizontal rotational components to estimate $\varphi$:

\begin{equation*}
    \varphi = - \arctan (\frac{\dot{\Omega}_n}{\dot{\Omega}_e})
\end{equation*}

The calculated $\varphi$ value from the inverse tangent ranges from $-90^\circ$ to $90^\circ$. We then convert it to the value between $0^\circ$ and $180^\circ$ by 

\begin{equation*}
\varphi = 
\begin{cases} \varphi, & \text{if } \varphi \geq 0 \\ \varphi + 180^\circ, & \mbox{if } \varphi < 0  \end{cases}.
\end{equation*}

Thus, the back azimuth $\varphi_{baz}$ (<font color='red'>with $180^\circ$ ambiguity</font>) is equal to

\begin{equation*}
\varphi_{baz} = \varphi \qquad   \text{or}  \qquad \varphi + 180
\end{equation*}

+ Removing $180^\circ$ ambiguity:

We rotate the east-west ($\Omega_E$) and north-south ($\Omega_N$) rotational components to radial ($\Omega_R$) and transverse ($\Omega_T$) components:

\begin{align*}
\begin{bmatrix} \dot{\Omega}_R  \\ \dot{\Omega}_T  \end{bmatrix} 
= \begin{bmatrix} -\sin \varphi_{baz} & -\cos \varphi_{baz}  \\ -\cos \varphi_{baz} & \sin \varphi_{baz}  \end{bmatrix} \begin{bmatrix} \dot{\Omega}_E  \\ \dot{\Omega}_N  \end{bmatrix}
=  A \frac{\omega^2}{v} \begin{bmatrix} 0  \\ \cos \xi  \end{bmatrix} \cos [ \omega (t - \mathbf{s} \cdot \mathbf{r}) + \phi + \pi/2].
\end{align*}

To remove the $180^{\circ}$, we can compare the transverse rotational component to vertical translational component. 

\begin{equation*}
\begin{bmatrix} \ddot{u}_Z  \\ \dot{\Omega}_T  \end{bmatrix}
= -  A \omega^2 \cos \xi \begin{bmatrix}  1  \\ -\frac{1}{v}  \end{bmatrix} \cos [ \omega (t - \mathbf{s} \cdot \mathbf{r}) + \phi + \pi/2].
\end{equation*}

If the trial $\varphi_{baz}$ with $180^\circ$ ambiguity is equal to the theorectical one, $\ddot{u}_z$ and $\dot{\Omega}_T$ will be negatively correlated. If $\ddot{u}_z$ and $\dot{\Omega}_T$ are positively correlated, we need to add $180^{\circ}$ to $\varphi_{baz}$.

In [ ]:
# Import 
%matplotlib inline 

#import matplotlib
from obspy import *
from obspy.core import AttribDict
from obspy.clients.filesystem import sds
from obspy.clients.fdsn import Client
import obspy.signal.array_analysis as AA
import obspy.signal.util as util
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
from obspy.signal.cross_correlation import correlate,xcorr_max
import matplotlib.colors as colors
from obspy.signal.trigger import trigger_onset,recursive_sta_lta
import matplotlib.cm as cm
import numpy as np
import csv
#import pickle
import scipy as sp
import scipy.odr as odr

In [ ]:
# simply estimating the slope of a function which is than used
# to calculate the baz
def fit_func(b, x):
    return b[0] * x + b[1]

def f_phi_cw_ray (phi_cw, rotrate_ne):
    """ Model function for orthogonal distance regression: vertical acceleration,
        horizontal components of rotation according to relation of 
        vertical acceleration - horizontal rotation for rayleigh waves:
    
    @type phi_cw: 2 component list or array where 
    @param phi_cw: phi_cw[0] is the backazimuth, phi_cw[1] is the velocity
    @type rot_ne: 2d numpy array of shape (2,n)
    @param rot_ne: rot_ne[0] is the North component of rotation
                  rot_ne[1] is the East component of rotation
    
    """
    phi = phi_cw[0]
    cw = phi_cw[1]
    rotrate_n = rotrate_ne[0]
    rotrate_e = rotrate_ne[1]

    # The rotated coordinate or rotation will be TRANSVERSAL to the vector pointing away from
    # source - receiver
    # the rotation components are always + 90 to the translation components thus the Transverse 
    # rotation is out of phase with the vertical translation
    rot_t = rotrate_n * np.sin(phi) - rotrate_e * np.cos( phi)
    return rot_t * cw

def f_phi_cw (phi_sl,acc_ne):
    """ Model function for Orthogonal Distance regression:

    @type phi_sl: 2 component list or array where
    @param phi_sl: phi_sl[0] is the backazimuth, phi_sl[1] is the slowness
    @type acc_ne: 2d numpy array of shape (2,n)
    @param acc_ne: acc_ne[0] is the North component of acceleration
                  acc_ne[1] is the East component of acceleration
    @return: numpy array , transversal acceleration in direction of
            the backazimuth multiplied by lowness divided by 2
    """
    acc_n = acc_ne[0]
    acc_e = acc_ne[1]
    acc_t = np.zeros(len(acc_n))
    phi = phi_sl[0]
    sl = phi_sl[1]
    acc_t = acc_n * np.sin(phi) - acc_e * np.cos(phi)
    return  acc_t*sl/2.


In [ ]:
def run(par):
    rad = par.rad 
    multi_f = par.multi_f
    freqmin = par.freqmin
    freqmax = par.freqmax
    periods_per_window = par.periods_per_window

    win_frac = par.win_frac
    exponent = par.exponent

    mask_value = par.mask_value

    df = par.df

    anti_trigger = par.anti_trigger
    #to avoid "noise"
    pro_trigger = par.pro_trigger
    tsta = par.tsta
    tlta = par.tlta
    thres1 = par.thres1
    thres2 = par.thres2
    pre = par.pre
    post = par.post

    cw_ini = par.cw_ini

    plot_ax = par.plot_ax 

    for dd in method:
        love = False
        flinn = False
        Z_vs_T = False
        ODR = False
# some setting for plotting    
        fmt = "png"
        dpi = 300
        if dd == "love":
            love = True;
            type_color = "red"
        elif dd == "flinn":
            flinn = True;
            type_color = "orange"
        elif  dd == "Z_vs_T":
            Z_vs_T = True
            type_color = "blue"
        elif  dd == "ODR":
            ODR = True
            type_color = "yellow"


        # The lowpass and highpass frequencies for bandpass filtering
        if multi_f == True:
           f_lower = []
           f_higher = []
           fcenter = freqmax
           #we try half octave band filtering
           while fcenter > freqmin:
              f_l = fcenter/(np.sqrt(np.sqrt(2.)))
              f_u = fcenter*(np.sqrt(np.sqrt(2.)))
              f_lower.append(f_l)
              f_higher.append(f_u)
              fcenter = fcenter/(np.sqrt(2.))

           f_lower = np.asarray(f_lower)
           f_higher = np.asarray(f_higher)
           print(f_lower)
        else:
           f_lower = [freqmin]
           f_higher = [freqmax]

        print("Lower frequency bounds :",f_lower)
        for i in range(len(f_lower)):
            res = []
            trace = Stream()
            trs = Stream()
            start = ot
            end = 0.
            fmin = f_lower[i]
            fmax = f_higher[i]

            print("now at: ",fmin," - ",fmax)
            while end < endy:
                tsz = []
                tsn = []
                tse = []
                coo = []
                first = True
                if (endy - start) > 3600:
                    end = start + 3600.
                else:
                    end = endy

                # get the data of the seismometer
                print(sstation)
                net,sta,loc,stream = sstation.split(".")
                inv_s = read_inventory("./stationxml/merapi_stationxml.xml")
                stats = client1.get_waveforms(network=net,station=sta,location=loc,channel=stream, starttime=start, endtime=end)
                stats.merge(method=1,fill_value="latest")
                stats.attach_response(inv_s)
                stats.sort()
                stats.reverse()

                stats.rotate(method='->ZNE', inventory=inv_s, components=['ZNE'])

                stats.resample(sampling_rate=df) #,no_filter=False)
                fs = stats[0].stats.sampling_rate
                print(stats)
                # stabilize it
                stats.detrend("linear")
                stats[0].filter('highpass',freq=0.01)
                stats[1].filter('highpass',freq=0.01)
                stats[2].filter('highpass',freq=0.01)

                # improve the bandwidth - caution this might cause also problems
                stats[0].remove_response(water_level=30, output="VEL")
                stats[1].remove_response(water_level=30, output="VEL")
                stats[2].remove_response(water_level=30, output="VEL")

                stats.detrend("simple")

                # get the data of the rotation sensor
                print(rstation)
                net,stat,loc,chan=rstation.split(".")
                rots = client2.get_waveforms(network=net,station=stat,location=loc,channel=chan,starttime=start,endtime=end)
                rots.merge(method=1,fill_value='latest')
                rots.resample(sampling_rate=df) #,no_filter=False) failes if df is larger than original sampling rate
                end2 = rots[0].stats.endtime
                rots.sort()
                rots.reverse()
                print(rots)

                rots.detrend("simple")
                rots.filter('highpass',freq=0.050)
                # to make some test we simply copy one trace
                rots.detrend("simple")
                print(rots)

                ###################################################################
                # Start of calculations ...
                ###################################################################

                # and the parameter periods_per_window
                if stats[0].stats.endtime > rots[0].stats.endtime:
                    end2 = rots[0].stats.endtime
                else:
                    end2 = stats[0].stats.endtime

                stats.trim(start,end2)
                rots.trim(start,end2)
                rots.detrend('simple')

                acc = stats.differentiate()
                acc.taper(type="cosine",max_percentage=0.01)
                rots.taper(type="cosine",max_percentage=0.01)
                rots.filter("bandpass",freqmin=fmin,freqmax=fmax)
                acc.filter("bandpass",freqmin=fmin,freqmax=fmax)

                # we perform a recursive sta/lta for detrigger the trace - just want to compute the background
                if anti_trigger:
                    cft = recursive_sta_lta(acc.select(component="Z")[0].data,int(tsta),int(tlta))
                    on_off = trigger_onset(cft,thres1,thres2)
                    for trig in on_off:
                        t1 = trig[0]
                        t2 = trig[1]
                        if t1-int(pre*df) > 0:
                           t1 -= int(pre*df)
                        else:
                           t1 = 0
                        if t2+int(post*df) < acc.select(component="Z")[0].stats.npts:
                           t2+=int(post*df)
                        else:
                           t2 = acc.select(component="Z")[0].stats.npts-1

                        rots.select(component="N")[0].data[t1:t2]=mask_value/10.
                        rots.select(component="E")[0].data[t1:t2]=mask_value/10.
                if pro_trigger:
                    cft = recursive_sta_lta(acc.select(component="Z")[0].data,int(tsta),int(tlta))
                    on_off = trigger_onset(cft,thres1,thres2)
                    for i,trig in enumerate (on_off):
                        t1 = trig[0]
                        t2 = trig[1]
                        if t1-int(pre*df) > 0:
                           t1 -= int(pre*df)
                        else:
                           t1 = 0
                        if t2+int(post*df) < acc.select(component="Z")[0].stats.npts:
                           t2+=int(post*df)
                        else:
                           t2 = acc[0].stats.npts-1
                        on_off[i,0] = t1
                        on_off[i,1] = t2

                        if i == 0:
                            rots.select(component="N")[0].data[0:t1]=mask_value/10.
                            rots.select(component="E")[0].data[0:t1]=mask_value/10.
                        elif i == len(on_off)-1:
                            rots.select(component="N")[0].data[t2:]=mask_value/10.
                            rots.select(component="E")[0].data[t2:]=mask_value/10.
                        else:
                            rots.select(component="N")[0].data[on_off[i-1,1]:t1]=mask_value/10.
                            rots.select(component="E")[0].data[on_off[i-1,1]:t1]=mask_value/10.


                trace += acc
                rots.detrend("linear")
                trs += rots

                print(rots.select(component="Z")[0].data.max())
                print(rots.select(component="Z")[0].data.min())

                nsamp = int(2*np.pi*periods_per_window*fs/(freqmax-freqmin))
                nstep = int(nsamp*win_frac)
                newstart = start
                offset = 0

                while (newstart + (nsamp)/fs) < end:
                    try:
                        rotn = sp.signal.detrend(rots.select(component="N")[0].data[offset:offset + nsamp],type="constant")
                        rote = sp.signal.detrend(rots.select(component="E")[0].data[offset:offset + nsamp],type="constant")
                        rotz = sp.signal.detrend(rots.select(component="Z")[0].data[offset:offset + nsamp],type="constant")
                        acc_n = acc.select(component="N")[0].data[offset:offset + nsamp]
                        acc_e = acc.select(component="E")[0].data[offset:offset + nsamp]
                        acc_z = acc.select(component="Z")[0].data[offset:offset + nsamp]

                        # in order to avoid problems with fliker noise of FOG we restirct the resolution
                        if love == True:
                            if mask_value > 0.:
                                rot_z = np.ma.masked_inside(rotz,-mask_value,mask_value)
                                masked = rot_z.mask
                                rot_z = rot_z[~masked]
                                rot_z=rot_z.compressed()
                            else:
                                rot_z = rotz


                            acc_ne = np.empty((2,rot_z.shape[0]))
                            if mask_value > 0.:
                                acc_ne[0] = acc_n[~masked]
                                acc_ne[1] = acc_e[~masked]
                            else:
                                acc_ne[0] = acc_n
                                acc_ne[1] = acc_e

                            if acc_ne.shape[1] >2:
                                # Calculate optimal direction with ODR
                                phi_0 =  np.random.random_sample()*2.*np.pi
                                cw_0 = np.random.random_sample()*500.

                                odr_data = odr.Data(acc_ne,rot_z)
                                odr_model = odr.Model(fcn = f_phi_cw)
                                odr_opti = odr.ODR(odr_data,odr_model,np.array([phi_0,1./cw_0]))
                                odr_opti.set_job(fit_type = 0, deriv = 1, var_calc = 2)
                                odr_output = odr_opti.run()
                                phi_opt = odr_output.beta[0]
                                cw_opt = 1/odr_output.beta[1]
                                err = odr_output.sum_square
                                err = odr_output.sum_square
                                delta = odr_output.sum_square_delta
                                eps = odr_output.sum_square_eps
                                inv_err = delta/np.sum(odr_output.y**2) + eps/np.sum(rot_z**2)
                                if np.sqrt(inv_err) < 1.:
                                    wght = (1. - np.sqrt(inv_err))**exponent
                                else:
                                    wght = 1e-4
                                rotsq = np.sum(rot_z**2)

                                az_error = np.arctan2(inv_err/2.,1.)
                                az_error = np.degrees(az_error)

                                # Correct optimal angle to be between 0 and 360 degrees
                                if cw_opt < 0.:
                                    cw_opt *= -1.
                                    phi_opt += np.pi
                                if cw_opt > 50.:
                                    if (phi_opt < 0. ) or (phi_opt > 2.*np.pi):
                                        mult = np.abs(np.floor(phi_opt/(2*np.pi)))
                                        if phi_opt < 0. :
                                            phi_opt += mult*2*np.pi
                                        elif phi_opt > 2*np.pi:
                                            phi_opt -= mult*2*np.pi
                                    if (phi_opt < 0.): phi_opt += np.pi
                                    if (phi_opt > 2*np.pi): phi_opt -= 2*np.pi

                                    #cross check if the quadrant is correct - be aware this is now in R,T ray based
                                    t_acc = acc_n*np.sin(phi_opt) - acc_e*np.cos(phi_opt)
                                    cc = correlate(t_acc,rot_z,shift=0)
                                    shift,mcor = xcorr_max(cc)
                                    if np.sign(mcor)<0:
                                        phi_opt += np.pi
                                        if (phi_opt > 2*np.pi): phi_opt -= 2*np.pi

                                    time = newstart - nsamp/(2.*fs)
                                    bazimuth = np.degrees(phi_opt)
                                    res.append([time.datetime,bazimuth,az_error,wght,mcor])
                                    res.append([time.datetime,bazimuth,az_error,wght,mcor])
                                    #print(".", end='', flush=True)
                            else:
                                print("|", end='', flush=True)

                        else:
                            if mask_value > 0.:
                                condition1 = (np.abs(rotn[:]) < mask_value) & (np.abs(rote[:]) < mask_value)
                                rot_n = np.ma.masked_array(rotn,mask=condition1)
                                rot_e = np.ma.masked_array(rote,mask=condition1)
                                acc_zd = np.ma.masked_array(acc_z,mask=condition1)
                                rot_n=rot_n.compressed()
                                rot_e=rot_e.compressed()
                                acc_zd=acc_zd.compressed()
                            else:
                                rot_n = rotn
                                rot_e = rote
                                acc_zd = acc_z

                            if len(rot_n)>2:
                                if flinn == False:
                                    if ODR:
                                        odr_data = odr.Data(-rot_e,rot_n)
                                        odr_model = odr.Model(fcn = fit_func)
                                        odr_opti = odr.ODR(odr_data,odr_model, beta0=np.array([0,0]))
                                        odr_output = odr_opti.run()

                                        az_sl = odr_output.beta[0]
                                        sl_error = odr_output.sd_beta[0]

                                    if Z_vs_T:

                                        # Calculate optimal direction with ODR
                                        rotrate_ne = np.empty((2,len(rot_n)))
                                        rotrate_ne[0]=rot_n
                                        rotrate_ne[1]=-rot_e

                                        phi_0 =  np.random.random_sample()*np.pi
                                        cw_0 = np.random.random_sample()*cw_ini

                                        # **************** initial guess cw_0, finding cw ****************************
                                        odr_data = odr.Data(rotrate_ne,acc_zd)
                                        odr_model = odr.Model(fcn = f_phi_cw_ray)
                                        odr_opti = odr.ODR(odr_data,odr_model, np.array([phi_0,cw_0]),maxit=10)
                                        odr_opti.set_job(fit_type = 0, deriv = 1, var_calc = 2)
                                        odr_output = odr_opti.run()

                                        az_sl = odr_output.beta[0]
                                        sl_error = odr_output.sd_beta[0]



                                    bazimuth = np.arctan2(az_sl,1.)
                                    az_error = np.arctan(sl_error)
                                    wght = (1.0 -  az_error/np.pi)**exponent
                                else:    #Flinn much faster!

                                    #movement of rotation rotN = sin(az) * rot; rotE = -cos(az) * rot
                                    data = np.zeros((2, len(rot_n)), dtype=np.float64)
                                    # East
                                    data[0, :] = -rot_e
                                    # North
                                    data[1, :] = rot_n
                                    covmat = np.cov(data)
                                    eigvec, eigenval, v = np.linalg.svd(covmat)
                                    #bazimuth = -(np.arctan(eigvec[1][0]/eigvec[0][0]))
                                    #the negative sign is due to the clockwise rotation arround E
                                    bazimuth = (np.arctan2(eigvec[0][0],eigvec[1][0]))
                                    az_error = np.arctan(eigenval[1] / eigenval[0])

                                    # Rectilinearity defined after Montalbetti & Kanasewich, 1970
                                    wght = (1.0 - np.sqrt(eigenval[1] / eigenval[0]))**exponent

                            # sign of Xcorr should make the distinguish of correct quadrant possibel! 
                                t_rot = rotn*np.sin(bazimuth) - rote*np.cos(bazimuth)

                                if rad:
                                    acc_r = -acc_n*np.cos(bazimuth) - acc_e*np.sin(bazimuth)
                                    cc = correlate(acc_r,t_rot,shift=0)
                                    shift,mcor = xcorr_max(cc)
                                    if np.sign(mcor)<0:
                                        bazimuth += np.pi
                                else:
                                    cc = correlate(acc_z,t_rot,shift=0)
                                    shift,mcor = xcorr_max(cc)
                                    if np.sign(mcor)>0:
                                        bazimuth += np.pi

                                if bazimuth > 2*np.pi:
                                    bazimuth -= 2*np.pi
                                if bazimuth < 0.:
                                    bazimuth += 2*np.pi

                                az_error= np.degrees(az_error)
                                bazimuth = np.degrees(bazimuth)


                            # Correct optimal angle to be between 0 and 360 degrees
                            #   res.append([newstart.timestamp-origin.timestamp+nsamp/(2.*fs),azimuth,az_error,wght])
                                time = newstart - nsamp/(2.*fs)
                                res.append([time.datetime,bazimuth,az_error,wght,mcor])
                            else:
                                print('-', end='', flush=True)
                    except:
                       print("-", end='', flush=True)

                    old_end = newstart + nsamp/fs
                    newstart += nstep/fs
                    offset += nstep
                start = old_end
                print('.', end='', flush=True)

            res = np.array(res)
            Tp,phi,err,weight,mcorr = res.T
            # save the output as csv file
            outfile = "%s/6C-%s_%s.%s_%s.csv"%(outdir,Tp[0],net,sta,dd)
            with open(outfile, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile, dialect='unix')
                writer.writerows(res)
            Tp = mdates.date2num(Tp)
            csvfile.close()
            mcorr = [0 if np.abs(a_) < 0.5 else a_ for a_ in mcorr]
            wght = weight*np.abs(mcorr)

     
    #############################################
    # Plotting .....
    #############################################
            hist_baz, bins_baz = np.histogram(phi, bins=int(360/2), range = (0,360), weights = wght, density = False)
            max_baz =  np.argmax(hist_baz)
            print(bins_baz[max_baz])


            trace.merge()
            trace.rotate('NE->RT',back_azimuth=bins_baz[max_baz])
            trs.merge()
            trs.rotate('NE->RT',back_azimuth=bins_baz[max_baz])

            print(trace)
            print(trs)
            xlocator = mdates.AutoDateLocator()

            fig = plt.figure()
            axbaz = fig.add_subplot(212)
            axtrace = fig.add_subplot(211,sharex=axbaz)
            if not love:
                axtrace.plot(trace[1].times("matplotlib"),trace.select(component="Z")[0].data*1e3,'k')
                axtrace.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
                axtrace.tick_params( axis='x',labelbottom='off')
                axtrace.set_ylabel('[mm/s**2]',color='k')
                for tl in axtrace.get_yticklabels():
                    tl.set_color('k')
                ax1=axtrace.twinx()
                tr_max = np.abs(trs.select(component="T")[0].data.max()*1e6)
                ax1.plot(trs[1].times("matplotlib"),trs.select(component="T")[0].data*1e6,'blue',alpha=0.2)
                ax1.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
                ax1.set_ylabel('[urad/s]',color='blue')
                ax1.set_ylim(-1*tr_max,tr_max)
                for tl in ax1.get_yticklabels():
                    tl.set_color('blue')

            else:
                axtrace.plot(trace[1].times("matplotlib"),trace.select(component="T")[0].data*1e3,'k')
                axtrace.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
                axtrace.tick_params( axis='x',labelbottom='off')
                axtrace.set_ylabel('[mm/s**2]',color='k')
                for tl in axtrace.get_yticklabels():
                    tl.set_color('k')
                ax1=axtrace.twinx()
                tr_max = np.abs(trs.select(component="Z")[0].data.max()*1e6)
                ax1.plot(trs[1].times("matplotlib"),trs.select(component="Z")[0].data*1e6,'blue',alpha=0.2)
                ax1.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
                ax1.set_ylabel('[urad/s]',color='blue')
                ax1.set_ylim(-1*tr_max,tr_max)
                for tl in ax1.get_yticklabels():
                    tl.set_color('blue')
            npts = trace[1].stats.npts
            axtrace.axvline(trace[1].times("matplotlib")[int(npts/6)], ls = "--",color="g")
            axtrace.axvline(trace[1].times("matplotlib")[int(npts/6)+nsamp], ls = "--",color="g")
    #      
    #      
            #axbaz.errorbar(Tp, phi,yerr=err,fmt='none',ecolor="blue",alpha=0.1)
            #axbaz.scatter(Tp, phi,c=mcorr,cmap='coolwarm',s=15)
            axbaz.scatter(Tp, phi,c=np.abs(mcorr),cmap='Reds',s=15)
            axbaz.set_ylabel('[deg]')
            axbaz.set_ylim(0,360)
            axbaz.set_xlim(Tp.min(),Tp.max())
            axbaz.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
            axbaz.xaxis.set_major_locator(xlocator)

            fig.autofmt_xdate()

            if pro_trigger and not love:
                plt.savefig('%s%s__%f_hist_trig_sv.%s'%(save_path,sstation,fmin,fmt), format=fmt, dpi=dpi)
            elif not pro_trigger and not love:
                plt.savefig('%s%s__%f_hist_sv.%s'%(save_path,sstation,fmin,fmt), format=fmt, dpi=dpi)
            elif not pro_trigger and love:
                plt.savefig('%s%s__%f_hist_sh.%s'%(save_path,sstation,fmin,fmt), format=fmt, dpi=dpi)
            else:
                plt.savefig('%s%s__%f_hist_trig_sh.%s'%(save_path,sstation,fmin,fmt), format=fmt, dpi=dpi)
            plt.show()

            phi = phi*2*np.pi/360
            hist_baz, bins_baz = np.histogram(phi, bins=int(360/2),
                                                            range = (0,2*np.pi),
                                                            weights = np.abs(wght),
                                                            density = True)


            cmap = cm.viridis
            bins_number = 180  # the [0, 360) interval will be subdivided into this
                            # number of equal bins
            bins = np.linspace(0.0, 2 * np.pi, bins_number + 1)
            angles = 2 * np.pi * phi/360.

            width = 2 * np.pi / bins_number
            ax = plt.subplot(1, 1, 1, projection='polar')
            ax.set_theta_zero_location("N")
            ax.set_theta_direction(-1)
            ax.set_yticklabels([])

            bars = ax.bar(bins_baz[:-1], hist_baz, width=width,bottom=0.0,alpha=.7,color=type_color)

            if love:
                plt.axis('off')

            if pro_trigger:
                if flinn:
                    plt.savefig('%s%s_%s_%f_polar_trig_sv.%s'%(save_path,sstation,dd,fmin,fmt), format=fmt, dpi=dpi,transparent=True)
                elif Z_vs_T:
                        plt.savefig('%s%s_%s_%f_polar_trig_sv.%s'%(save_path,sstation,dd,fmin,fmt), format=fmt, dpi=dpi,transparent=True)
                elif love:
                    plt.savefig('%s%s_%s_%f_polar_trig_sh.%s'%(save_path,sstation,dd,fmin,fmt), format=fmt, dpi=dpi,transparent=True)
                elif ODR:
                    plt.savefig('%s%s_%s_%f_polar_trig_sv.%s'%(save_path,sstation,dd,fmin,fmt), format=fmt, dpi=dpi,transparent=True)
            else:
                if flinn:
                    plt.savefig('%s%s_%s_%f_polar_sv.%s'%(save_path,sstation,dd,fmin,fmt), format=fmt, dpi=dpi,transparent=True)
                elif Z_vs_T:
                    plt.savefig('%s%s_%s_%f_polar_sv.%s'%(save_path,sstation,dd,fmin,fmt), format=fmt, dpi=dpi,transparent=True)
                elif love:
                    plt.savefig('%s%s_%s_%f_polar_sh.%s'%(save_path,sstation,dd,fmin,fmt), format=fmt, dpi=dpi,transparent=True)
                elif ODR:
                    plt.savefig('%s%s_%s_%f_polar_sv.%s'%(save_path,sstation,dd,fmin,fmt), format=fmt, dpi=dpi,transparent=True)

            plt.show()
            plt.close("all")
                                                                                                                                                     

 
## Methods available: flinn, love, ODR, Z_vs_T
flinn (Rayleigh or SV): rotational components only using Flinns algorithm (covariance and SVD)\
love (or SH): searching for the  direction by optimizing linear relationship using ODR $a_T$ vs $\Omega_Z$\
ODR (Rayleigh or SV): searching for the direction by optimizing linear relationship using ODR $\Omega_N$ vs $-\Omega_E\$
Z_vs_T: similar to love but using the equations and components for $\Omega_T$ vs $a_Z$

## Parameters to be set
sstation: translational motion seed id (Net.Station.location.HH?)
rstation: rotational motion seed id (Net.Station.location.HJ?)
outdir: where to write results (csv file)
save_path: where to store produced figures
method: ["flinn","love","Z_vs_T","ODR"]
ot=start: start time (UTCDateTime)
endy: overall end time (UTCDateTime)
### where to get the data from: 
client1: translational motion (e.g. filesystem in SDS format)
client2: rotational motion (e.g. filesystem in SDS format)
### if transverse rotation should be compared with az or ar
rad: if true we assume very steeply incident SV-type waves and compare $\Omega_T$ to $a_R$
### in frequency HOBs or one single band
multi_f: if true we are performing the analysis in half octave bandpasses 
freqmin: lower corner frequency
freqmax: upper corner frequency
periods_per_window: how many periods long (freqmin) is the analysis window to be set\
                    $1 = 2*\pi/freqmin$ 

win_frac: step width of sliding window
exponent: weighting factor's exponent

### to mask values below the self noise level of the rotation sensor
mask_value: in rad/s negative values ignores

### new sampling rate
df: sampling rate in Hz

### to avoid earthquakes
anti_trigger: if True the triggered windows are NOT used for the analysis

### to avoid "noise"
pro_trigger: if True only the triggered windows are used 
### trigger settings for recursive STA/LTA trigger
tsta: length of STA window (seconds)
tlta: lenght of LTA window (seconds)
thres1: threshold trigger "on"
thres2: threshold trigger "off"
pre: pre trigger buffer (seconds)
post: post trigger buffer (seconds)

cw_ini: initial slope for ODR (m/s)

plot_ax: if False axes in the polar histogram plot are NOT drawn


In [ ]:
sstation = "XM.GRW0..BH?"
rstation = "XM.GRW0.10.BJ?"
outdir = "./6C-steer/"
save_path = "./Figures/"

method = ["flinn","love"] #,"Z_vs_T","ODR"]
#ot=start= UTCDateTime("2001-10-19T14:20")
#endy = UTCDateTime("2001-10-19T16:30")
#ot=start=UTCDateTime("2001-11-08T14:00") 
#endy = UTCDateTime("2001-11-08T14:30")
#ot=start=UTCDateTime("2001-10-26T05:00") 
#endy = UTCDateTime("2001-10-26T05:20")
ot=start=UTCDateTime("1998-11-04T22:29") 
endy = UTCDateTime("1998-11-04T22:40")
# Data Base
client1 = sds.Client(sds_root = "./data_sds")
client2 = sds.Client(sds_root = "./data_adr")

#if transverse rotation should be compared with az or ar
par = AttribDict()
par.rad = False
# in frequency HOBs or one single band
par.multi_f = False
par.freqmin= 0.10
par.freqmax = 2.0
par.periods_per_window= 1

par.win_frac = 0.1
par.exponent = 0.3

# to mask values below the self noise level of the 
# rotation sensor negative values ignores
par.mask_value = -1.e-9

#new sampling rate
par.df = 50.

# to avoid earthquakes
par.anti_trigger = False

#to avoid "noise"
par.pro_trigger = False
par.tsta = int(0.5*par.df)
par.tlta = int(10.*par.df)
par.thres1=3.5
par.thres2=0.5
par.pre = 10.
par.post = 10.

par.cw_ini = 1000.

par.plot_ax = False
run(par)


## More advanced plotting comes now ...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from obspy import read_inventory, UTCDateTime
import cartopy.crs as ccrs
import csv

from math import asin, atan2, cos, degrees, radians, sin

In [ ]:
# helper function for qiver
def get_point_at_distance(lat1, lon1, d, bearing, R=6371):
    """
    lat: initial latitude, in degrees
    lon: initial longitude, in degrees
    d: target distance from initial
    bearing: (true) heading in degrees
    R: optional radius of sphere, defaults to mean radius of earth

    Returns new lat/lon coordinate {d}km from initial, in degrees
    """
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    a = radians(bearing)
    lat2 = asin(sin(lat1) * cos(d/R) + cos(lat1) * sin(d/R) * cos(a))
    lon2 = lon1 + atan2(
        sin(a) * sin(d/R) * cos(lat1),
        cos(d/R) - sin(lat1) * sin(lat2)
    )
    return (degrees(lat2), degrees(lon2))

In [ ]:
# read in topo data (on a regular lat/lon grid)
# (srtm data from: http://srtm.csi.cgiar.org/)
srtm = np.loadtxt("./Merapi_Cont/output_SRTMGL1.asc", skiprows=6)
srtm = np.flipud(np.asarray(srtm))

min_lon = 110.41
max_lon = 110.48
min_lat = -7.58
max_lat = -7.51

# origin of data grid as stated in srtm data file header
# create arrays with all lon/lat values from min to max and
lats = np.linspace(-7.686824838501053,-7.410920503923094,srtm.shape[0])
lons = np.linspace(110.28845429420473, 110.64332127571106,srtm.shape[1])

# create Basemap instance with Mercator projection
# we want a slightly smaller region than covered by our srtm data
proj = ccrs.TransverseMercator(
    central_latitude=np.mean((min_lat, max_lat)),
      central_longitude=np.mean((min_lon, max_lon)))
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection=proj)
ax.set_extent((110.41,110.48, -7.58, -7.51), crs=proj)

# create grids and compute map projection coordinates for lon/lat grid
x, y = np.meshgrid(lons, lats)

# Make contour plot
cs = ax.contour(x, y, srtm, 40, colors="k", alpha=0.3, transform=proj)
# https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.mpl.geoaxes.GeoAxes.html#cartopy.mpl.geoaxes.GeoAxes.gridlines
ax.gridlines(draw_labels=True, color='k', linestyle='--')

stations_f = []
t1 = UTCDateTime("2001-01-01")
inv = read_inventory("./stationxml/merapi_stationxml.xml")
for stat in inv[0]:
    stat_id = "XM.%s" % stat.get_contents()['channels'][0]
    coo = inv.get_coordinates(stat_id)
    ll = [stat.code, coo['longitude'], coo['latitude'], coo['elevation']]
    stations_f.append(ll)



for name, lon, lat, elev in stations_f:
    ax.plot([lon], [lat], "rv", markersize=5, transform=proj)
    try:
        path = "./6C-steer/6C-1998-11-04 22:28:58.350000_XM.%s_flinn.csv"%name
        baz = []
        mcorr = []
        with open(path, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                baz.append(float(row[1]))
                mcorr.append(float(row[4]))

        baz = np.asarray(baz)
        mcorr = np.asarray(mcorr)
        for i in range(len(baz)):
            if np.abs(mcorr[i]) > 0.6:
                y,x = get_point_at_distance(lat, lon, 0.1, baz[i])
                dy = (y - lat)*10000
                dx = (x - lon)*10000
                ax.quiver(lon,lat,dx,dy,angles="xy",scale=0.1,color="green",alpha=np.abs(mcorr[i])/10) #,transform=proj)
    except:
        continue

fig.savefig("merapi.png", dpi=300)
plt.show()
                                                                                